In [1]:
import os, sys, pickle, random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.cm as cm
from tqdm import tqdm
from pathlib import Path
import mne
if os.getcwd().split("/")[-1] != 'BENDR-XAI': os.chdir("../")
from notebooks.utils import *

In [2]:
data = np.load("data/HCPMMP1_combined_8.0_12.0_220503_280423.npy", allow_pickle=True).item()

In [3]:
count = 0
for name in data.keys():
    count += len(data[name]['annotations']['T0'])

In [4]:
count / 148

150.79054054054055

In [31]:
channels_order = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']
raw = read_TUH_edf("/scratch/s194260/tuh_eeg/aaaaaado_s004_t002.edf")

In [40]:
 channel_map = {
        'EEG C3-REF': 'C3', 'EEG P4-REF': 'P4', 'EEG T5-REF': 'P7', 'EEG F8-REF': 'F8', 'EEG F7-REF': 'F7',
        'EEG C4-REF': 'C4', 'EEG PZ-REF': 'Pz', 'EEG FP2-REF': 'Fp2', 'EEG F4-REF': 'F4', 'EEG F3-REF': 'F3',
        'EEG T6-REF': 'P8', 'EEG CZ-REF': 'Cz', 'EEG O2-REF': 'O2', 'EEG O1-REF': 'O1', 'EEG T2-REF': 'FT8',
        'EEG T1-REF': 'FT7', 'EEG T4-REF': 'T8', 'EEG P3-REF': 'P3', 'EEG FZ-REF': 'Fz', 'EEG T3-REF': 'T7',
        'EEG FP1-REF': 'Fp1', 'EEG C4-LE': 'C4', 'EEG P3-LE': 'P3', 'EEG FZ-LE': 'Fz', 'EEG F3-LE': 'F3',
        'EEG FP1-LE': 'Fp1', 'EEG T6-LE': 'P8', 'EEG CZ-LE': 'Cz', 'EEG F8-LE': 'F8', 'EEG O1-LE': 'O1',
        'EEG PZ-LE': 'Pz', 'EEG C3-LE': 'C3', 'EEG FP2-LE': 'Fp2', 'EEG O2-LE': 'O2', 'EEG FP1-LE': 'Fp1',
        'EEG F7-LE': 'F7', 'EEG T1-LE': 'FT7', 'EEG T2-LE': 'FT8', 'EEG P4-LE': 'P4', 'EEG T4-LE': 'T8',
    }

# Filter the channel_map to include only the channels present in the raw data
channel_map_sub = {k: v for k, v in channel_map.items() if k in raw.ch_names}

# Standardize the raw data
mne.datasets.eegbci.standardize(raw)

# Rename the channels using the filtered channel_map
raw = raw.rename_channels(channel_map_sub)

In [33]:
subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()

In [34]:
fwd = get_fwd(raw.info, trans, src_path, bem_path)

In [38]:
montage = mne.channels.make_standard_montage('standard_1020')

    # Set the montage for the raw data, ignoring missing channels
raw = raw.set_montage(montage)

In [23]:
raw = raw.reorder_channels(channels_order)


ValueError: 'T5' is not in list

In [24]:
raw.ch_names

['Fp1',
 'Fp2',
 'F3',
 'F4',
 'C3',
 'C4',
 'P3',
 'P4',
 'O1',
 'O2',
 'F7',
 'F8',
 'T7',
 'T8',
 'P7',
 'P8',
 'Fz',
 'Cz',
 'Pz']

In [2]:
# Make a list of path to all edf files in the subfolders of /nobackup/tsal-tmp/tuh_eeg
# files = []

# with tqdm() as pbar:
#     for root, dirs, file in os.walk("/nobackup/tsal-tmp/tuh_eeg"):
#         for name in file:
#             if name.endswith(".edf"):
#                 files.append(os.path.join(root, name))
#                 pbar.update(1)

# Save list of files as pickle
# import pickle
# with open('tuh_files.pkl', 'wb') as f:
#     pickle.dump(files, f)
    
# Load list of files from pickle
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)
    
# random.shuffle(files)

In [3]:
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)

# Copy all files in tuh_final_selected.txt to data/tuh_eeg/
# with open('tuh_final_selected.txt', 'r') as f:
#     selected = f.read().splitlines()

# selected = [file.split("/")[-1] for file in selected]

# Delete all files in /scratch/s194260/tuh_eeg/ that are not in selected
# for file in os.listdir('/scratch/s194260/tuh_eeg/'):
#     if file not in selected:
#         os.remove('/scratch/s194260/tuh_eeg/' + file)